# Atomic OpenCode Prompt Executor

This notebook serves as an atomic agent execution unit for OpenCode.
It accepts a prompt via Papermill parameters, executes it using the OpenCode CLI,
and outputs the result to stdout for further processing.

## Usage with Papermill:
```bash
papermill opencode_prompt.ipynb output.ipynb -p prompt "Generate a riddle about programming"
```

## Parameters:
- `prompt`: The prompt to send to OpenCode (required)
- `model`: Optional model specification (e.g., "opencode/grok-code")
- `session_id`: Optional session ID to continue a conversation
- `additional_args`: Optional additional OpenCode CLI arguments
- `output_format`: How to format the output (default: "text")


In [ ]:
# Papermill Parameters - These will be injected by Papermill
# Default values for standalone testing

# Required parameter
prompt="Create a simple riddle about artificial intelligence"

# Optional parameters
model=""                    # e.g., "opencode/grok-code" or "opencode/big-pickle"
session_id=""               # Continue existing session
additional_args=""          # Additional CLI arguments
output_format="text"        # Output format: "text", "json", "markdown"
timeout="300"               # Timeout in seconds
verbose="false"             # Enable verbose output

In [ ]:
#!/bin/bash
# Validate required parameters
if [ -z "$prompt" ]; then
    echo "ERROR: 'prompt' parameter is required" >&2
    echo "Usage: papermill opencode_prompt.ipynb output.ipynb -p prompt 'Your prompt here'" >&2
    exit 1
fi

echo "📝 OpenCode Atomic Agent Execution"
echo "================================="
echo "Prompt: $prompt"
echo "Model: ${model:-'default'}"
echo "Session ID: ${session_id:-'new session'}"
echo "Timeout: ${timeout}s"
echo "================================="

In [ ]:
#!/bin/bash
# Build OpenCode command with parameters
cmd="opencode run"

# Add optional parameters if provided
if [ -n "$model" ]; then
    cmd="$cmd --model \"$model\""
fi

if [ -n "$session_id" ]; then
    cmd="$cmd --session \"$session_id\""
fi

# Add any additional arguments
if [ -n "$additional_args" ]; then
    cmd="$cmd $additional_args"
fi

# Add the prompt (always last)
cmd="$cmd \"$prompt\""

if [ "$verbose" = "true" ]; then
    echo "🔧 Executing command: $cmd"
    echo "⏱️  Timeout: ${timeout}s"
    echo ""
fi

echo "🚀 Starting OpenCode execution..."
echo ""

In [ ]:
#!/bin/bash
# Execute OpenCode with timeout and capture output
echo "🤖 OpenCode Response:"
echo "==================="

# Create a temporary file for the response
TEMP_FILE=$(mktemp)
EXIT_CODE=0

# Execute with timeout
if timeout "$timeout" bash -c "$cmd" > "$TEMP_FILE" 2>&1; then
    EXIT_CODE=$?
else
    EXIT_CODE=$?
    if [ $EXIT_CODE -eq 124 ]; then
        echo "ERROR: OpenCode execution timed out after ${timeout}s" >&2
        rm -f "$TEMP_FILE"
        exit 1
    fi
fi

# Check if execution was successful
if [ $EXIT_CODE -ne 0 ]; then
    echo "ERROR: OpenCode execution failed with exit code $EXIT_CODE" >&2
    echo "Output:" >&2
    cat "$TEMP_FILE" >&2
    rm -f "$TEMP_FILE"
    exit 1
fi

# Display the response
cat "$TEMP_FILE"

# Clean up
rm -f "$TEMP_FILE"

In [ ]:
#!/bin/bash
# Output formatting and metadata
echo ""
echo "==================="
echo "✅ Execution completed successfully"
echo "📊 Execution metadata:"
echo "   - Prompt: $prompt"
echo "   - Model: ${model:-'default'}"
echo "   - Session: ${session_id:-'new'}"
echo "   - Timestamp: $(date -Iseconds)"
echo "   - Format: $output_format"

# Output format-specific processing
case "$output_format" in
    "json")
        echo "   - Output: JSON formatted response above"
        ;;
    "markdown")
        echo "   - Output: Markdown formatted response above"
        ;;
    *)
        echo "   - Output: Plain text response above"
        ;;
esac

echo "🎯 Atomic agent execution complete!"

In [ ]:
#!/bin/bash
# Final output for stdout (for pipeline integration)
# This can be captured by other processes
echo ""
echo "=== PAPERMILL_OUTPUT_BEGIN ==="
echo "EXECUTION_STATUS=SUCCESS"
echo "PROMPT=$prompt"
echo "MODEL=${model:-'default'}"
echo "SESSION=${session_id:-'new'}"
echo "TIMESTAMP=$(date -Iseconds)"
echo "=== PAPERMILL_OUTPUT_END ==="